In [1]:
! pip3 install cassandra-driver

    100% |################################| 491kB 669kB/s ta 0:00:011
  Running setup.py bdist_wheel for cassandra-driver ... done
  Stored in directory: /root/.cache/pip/wheels/b2/22/a0/77ab16988dfe73cc3e42141774606b3e41eb3f02225e22ba98
Successfully built cassandra-driver


In [24]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [25]:
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')

In [26]:
cluster = Cluster(['127.0.0.1'],port=9042,auth_provider=auth_provider)

In [27]:
session = cluster.connect()

# replication 설정 

## class가 SimpleStrategy인 경우<br>
"class" : "SimpleStrategy" - 'replication_factor' : N <br>
- 전체 클러스터에 동일한 복제 요소를 할당
- 평가 및 단일 데이터 센터 테스트 및 개발 환경에만 사용

## class가 'NetworkTopologyStrategy인 경우<br>
'NetworkTopologyStrategy'  -  'datacenter_name' : N <br>
- 쉼표로 구분 된 목록의 각 데이터 센터에 복제 요소를 할당
- 프로덕션 환경 및 다중 DC 테스트 및 개발 환경에서 사용
- 데이터 센터 이름은 snitch DC 이름과 일치

#### 멱등성 : 출력을 다시 입력에 넣어도 같은 출력   -> keyspace가 있으면 안 만든다

In [28]:
def create_keyspace(session,keyspace):
    session.execute(""" \
                    CREATE KEYSPACE IF NOT EXISTS {} \
                    WITH replication = {{'class': 'SimpleStrategy','replication_factor': '2' }} \
                    """.format(keyspace))

In [40]:
create_keyspace(session,'example')

In [41]:
session.set_keyspace('example')
# or you can do this instead
#session.execute('USE users')

In [126]:
session.execute("""
        CREATE TABLE IF NOT EXISTS mytable (
            thekey text,
            col1 text,
            col2 text,
            PRIMARY KEY (thekey)
        )
        """)

In [127]:
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel

query = SimpleStatement("""
        INSERT INTO mytable (thekey, col1, col2)
        VALUES (%(key)s, %(a)s, %(b)s)
        """, consistency_level=ConsistencyLevel.ONE)

In [128]:
prepared = session.prepare("""
        INSERT INTO mytable (thekey, col1, col2)
        VALUES (?, ?, ?)
        """)

In [130]:
for i in range(10):
        if (i % 2) == 1:
            session.execute(query, dict(key="key%d" % i, a='a', b='b'))
        else:
            session.execute(prepared, ("key%d" % i, 'alpha', 'beta'))

In [131]:
future = session.execute_async("SELECT * FROM mytable")
rows = future.result()

In [132]:
for row in rows:
    print("\t".join(row))

key5	a	b
key6	alpha	beta
key0	alpha	beta
key1	a	b
key9	a	b
key4	alpha	beta
key8	alpha	beta
key3	a	b
key7	a	b
key2	alpha	beta


| Level | Description | Usage |
|:---|:---|:---|
|ALL | A write must be written to the commit log and memtable on all replica nodes in the cluster for that partition. | Provides the highest consistency and the lowest availability of any other level.|
|EACH_QUORUM | Strong consistency. A write must be written to the commit log and memtable on a quorum of replica nodes in each datacenter. | Used in multiple datacenter clusters to strictly maintain consistency at the same level in each datacenter. For example, choose this level if you want a write to fail when a datacenter is down and the QUORUM cannot be reached on that datacenter.|
|QUORUM | A write must be written to the commit log and memtable on a quorum of replica nodes across all datacenters. | Used in either single or multiple datacenter clusters to maintain strong consistency across the cluster. Use if you can tolerate some level of failure.|
|LOCAL_QUORUM | Strong consistency. A write must be written to the commit log and memtable on a quorum of replica nodes in the same datacenter as the coordinator. Avoids latency of inter-datacenter communication. | Used in multiple datacenter clusters with a rack-aware replica placement strategy, such as NetworkTopologyStrategy, and a properly configured snitch. Use to maintain consistency locally (within the single datacenter). Can be used with SimpleStrategy.|
|ONE | A write must be written to the commit log and memtable of at least one replica node. | Satisfies the needs of most users because consistency requirements are not stringent.|
|TWO | A write must be written to the commit log and memtable of at least two replica nodes. | Similar to ONE.|
|THREE | A write must be written to the commit log and memtable of at least three replica nodes. | Similar to TWO.|
|LOCAL_ONE | A write must be sent to, and successfully acknowledged by, at least one replica node in the local datacenter. | In a multiple datacenter clusters, a consistency level of ONE is often desirable, but cross-DC traffic is not. LOCAL_ONE accomplishes this. For security and quality reasons, you can use this consistency level in an offline datacenter to prevent automatic connection to online nodes in other datacenters if an offline node goes down.|
|ANY | A write must be written to at least one node. If all replica nodes for the given partition key are down, the write can still succeed after a hinted handoff has been written. If all replica nodes are down at write time, an ANY write is not readable until the replica nodes for that partition have recovered. | Provides low latency and a guarantee that a write never fails. Delivers the lowest consistency and highest availability.|

In [133]:
query = SimpleStatement("""
        INSERT INTO mytable (thekey, col1)
        VALUES (%(key)s, %(a)s)
        """, consistency_level=ConsistencyLevel.ONE)

In [134]:
prepared = session.prepare("""
        INSERT INTO mytable (thekey, col2)
        VALUES (?, ?)
        """)

In [138]:
for i in range(100):
    if (i % 2) == 1:
        session.execute(query, dict(key="key%d" % i, a='aaaa'))
    else:
        session.execute(prepared, ("key%d" % i, 'bbbbbb'))

In [125]:
session.execute("""drop table mytable""")